In [1]:
from processing_framework import Sequencer, GetPaths, LoadImages, ShortenPaths, ChangeImageMode
from processing_framework import FilterImages, FilterByColorsNumber
import numpy as np

In [2]:
seq = Sequencer([GetPaths(recursive=True,
                          allowed_extensions=['.gif', '.png', '.jpg', '.jpeg'],
                          banned_keywords=['back']),
                 LoadImages(),
                 ChangeImageMode(),
                 FilterImages(mode='RGBA', filter_paths=True),
                 FilterByColorsNumber(min_colors=2, max_colors=32),
                 #ResizeImages((96, 96), 'lanczos'),
                 #PadImages(to_resolution=(128, 128), color=255),
                 #ImagesToArrays(),
                 #StandarizeBackgroundByAlpha((255, 255, 255), binarize_alpha=True),
                 #ArraysToImages(),
                 ShortenPaths(delete_patterns=[r'raw\/'], delete_last_segment=True),
                 #SaveImages('processed/pokemon_all', use_paths=True, clean_destination_dir=True)
                 ])

In [3]:
path = 'raw/pokemon'
data = seq(paths=[path])
print('\nOverall time:', seq.time_taken)

Step 1/6 (GetPaths) in progress
dict_keys(['paths'])
Found 27014 paths
Step 1/6 (GetPaths) completed, time elapsed: 2.195099353790283 seconds

Step 2/6 (LoadImages) in progress
dict_keys(['paths'])
Could not open image: raw/pokemon\10180.png
Loaded 27012 images
Step 2/6 (LoadImages) completed, time elapsed: 82.6601173877716 seconds

Step 3/6 (ChangeImageMode) in progress
dict_keys(['images', 'paths'])
Mode of 15873 images changed
Step 3/6 (ChangeImageMode) completed, time elapsed: 2.1957099437713623 seconds

Step 4/6 (FilterImages) in progress
dict_keys(['paths', 'images'])
Filtered 2449 images out of 27012. 24563 images remaining
27013
24563
24563
Step 4/6 (FilterImages) completed, time elapsed: 9.730723142623901 seconds

Step 5/6 (FilterByColorsNumber) in progress
dict_keys(['images', 'paths'])


AttributeError: reshape

In [ ]:
unique_paths = set(data['paths'])

In [ ]:
def get_number_of_colors(img):
    n_colors = len(np.unique(img.reshape(-1, img.shape[-1]), axis=0))
    return n_colors

In [ ]:
stats = {}
for unique_path in unique_paths:
    images = [img for img, path in zip(data['images'], data['paths']) if path == unique_path]
    
    arrays = [np.array(img) for img in images]
    d = {}
    d['count'] = len(images)
    modes, counts = np.unique([img.mode for img in images], return_counts=True)
    d['modes'] = list(zip(modes, counts))
    
    '''
    nr_of_colors = [get_number_of_colors(arr) for arr in arrays]
    d['colors_mean'] = np.mean(nr_of_colors)
    d['colors_Q1'] = np.quantile(nr_of_colors, 0.01)
    d['colors_Q10'] = np.quantile(nr_of_colors, 0.1)
    d['colors_median'] = np.median(nr_of_colors)
    d['colors_Q90'] = np.quantile(nr_of_colors, 0.9)
    d['colors_Q99'] = np.quantile(nr_of_colors, 0.99)
    '''
    
    res, counts = np.unique([img.size for img in images], return_counts=True, axis=0)
    d['sizes'] = list(zip(res, counts))
    ratios, counts = np.unique([img.size[0] / img.size[1] for img in images], return_counts=True)
    d['ratios'] = list(zip(ratios, counts))
    stats[unique_path] = d
    

In [ ]:
THRESHOLD = 5.0

keys = sorted(list(stats.keys()), key=lambda x: stats[x]['colors_mean'])

for path in keys:
    print(f'{path}:')
    for key in stats[path]:
        l = stats[path][key]
        if isinstance(l, list):
            counts = [x[-1] for x in l]
            cnt = sum(counts)
            new_l = []
            other_cnt = 0
            for x in l:
                new_x = (x[0], round(x[1] / cnt * 100.0, 2))
                if new_x[1] >= THRESHOLD:
                    new_l.append(new_x)
                else:
                    other_cnt += x[1]
                    
            if other_cnt > 0:
                new_l.append(('other', round(other_cnt / cnt * 100.0, 2)))
            l = sorted(new_l, key=lambda x: x[1], reverse=True)
            l = [(x[0], str(x[1]) + '%') for x in l]
            
            print(f'\t{key}: {l}')
        else:
            print(f'\t{key}: {l}')

In [ ]:
sum([stats[key]['count'] for key in stats.keys()])

In [ ]:
24900